# 表示文本

如果我們想用神經網絡解決自然語言處理 (NLP) 任務，我們需要某種方式將文本表示為張量。計算機已經將文本字符表示為數字，使用 ASCII 或 UTF-8 等編碼映射到屏幕上的字體。

![顯示將字符映射到 ASCII 和二進製表示的圖表的圖像](./img/ascii-character-map.png)

我們了解每個字母 **代表** 的含義，以及所有字符如何組合在一起形成句子的單詞。但是，計算機本身並沒有這樣的理解，神經網絡必須在訓練時學習其含義。

因此，我們可以在表示文本時使用不同的方法：
* **字符級表示**，當我們通過將每個字符視為一個數字來表示文本時。鑑於我們的文本語料庫中有 $C$ 不同的字符，單詞 *Hello* 將由 $5\times C$ 張量表示。每個字母將對應於 one-hot 編碼中的一個張量列。
* **詞級表示**，其中我們創建了文本中所有單詞的**詞彙表**，然後使用 one-hot 編碼表示單詞。這種方法在某種程度上更好，因為每個字母本身並沒有多大意義，因此通過使用更高級別的語義概念——單詞——我們簡化了神經網絡的任務。然而，鑑於字典很大，我們需要處理高維稀疏張量。

讓我們從安裝一些我們將在本模塊中使用的必需 Python 包開始。

In [1]:
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

     |████████████████████████████████| 24.2 MB 4.1 MB/s            
     |████████████████████████████████| 1.4 MB 6.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 15.1 MB 6.0 MB/s            
     |████████████████████████████████| 40.3 MB 3.7 MB/s            
     |████████████████████████████████| 2.2 MB 3.4 MB/s            
     |████████████████████████████████| 119.6 MB 4.3 MB/s            
     |████████████████████████████████| 1.5 MB 27.6 MB/s            
     |████████████████████████████████| 1.6 MB 4.9 MB/s            
     |████████████████████████████████| 13.2 MB 5.2 MB/s            
     |████████████████████████████████| 1.9 MB 5.4 MB/s            
     |████████████████████████████████| 58 kB 5.2 MB/s            
     |████████████████████████████████| 288 kB 6.6 MB/s            
     |████████████████████████████████| 76 kB 4.7 MB/s            
     |████████████████████████████████| 895 kB 5.4 MB/s            
  

# 文本分類任務

在本模塊中，我們將從基於 **AG_NEWS** 數據集的簡單文本分類任務開始，該任務將新聞標題分類為 4 個類別之一：World、Sports、Business 和 Sci/Tech。 該數據集內置於 [`torchtext`](https://github.com/pytorch/text) 模塊中，因此我們可以輕鬆訪問它。

In [2]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

train.csv: 29.5MB [00:01, 15.0MB/s]
test.csv: 1.86MB [00:00, 4.46MB/s]


這裡，`train_dataset` 和 `test_dataset` 包含迭代器，它們分別返回標籤對（類別數）和文本，例如：

In [3]:
next(train_dataset)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

因此，讓我們打印出數據集中的前 10 個新標題：

In [4]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace ba

Because datasets are iterators, if we want to use the data multiple times we need to convert it to list:

In [5]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

現在我們需要將文本轉換為可以表示為張量的**數字**。 如果我們想要詞級表示，我們需要做兩件事：
* 使用 **tokenizer** 將文本拆分為 **tokens**
* 構建這些標記的 **詞彙**。

In [6]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [7]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

使用詞彙表，我們可以輕鬆地將標記化的字符串編碼為一組數字：

In [8]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95812


[283, 2321, 5, 337, 19, 1301, 2357]

## 詞袋文本表示

因為單詞代表意義，有時我們可以通過查看單個單詞來理解文本的含義，而不管它們在句子中的順序如何。 例如，在對新聞進行分類時，像*weather*、*snow* 這樣的詞可能表示*weather forecast*，而像*stocks*、*dollar* 這樣的詞將計入*financial news*。

**詞袋**（BoW）向量表示是最常用的傳統向量表示。 每個詞都鏈接到一個向量索引，向量元素包含一個詞在給定文檔中出現的次數。

![圖像顯示了詞袋向量表示如何在內存中表示。](./img/bag-of-words-example1.png)

> **注意**：您還可以將 BoW 視為文本中單個單詞的所有單熱編碼向量的總和。

下面是如何使用 Scikit Learn python 庫生成詞袋表示的示例：

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()



array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

要從 AG_NEWS 數據集的向量表示中計算詞袋向量，我們可以使用以下函數：

In [10]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([0., 0., 2.,  ..., 0., 0., 0.])


> **注意：** 這裡我們使用全局`vocab_size` 變量來指定詞彙表的默認大小。 由於通常詞彙量非常大，我們可以將詞彙量限制為最常用的詞。 嘗試降低 `vocab_size` 值並運行下面的代碼，看看它如何影響準確性。 您應該期待一些準確度下降，但不會顯著下降，以代替更高的性能。

## 訓練 BoW 分類器

現在我們已經學會瞭如何構建文本的詞袋表示，讓我們在它之上訓練一個分類器。 首先，我們需要以這種方式轉換我們的訓練數據集，將所有位置向量表示轉換為詞袋表示。 這可以通過將 `bowify` 函數作為 `collate_fn` 參數傳遞給標準 Torch `DataLoader` 來實現：

In [11]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

現在讓我們定義一個包含一個線性層的簡單分類器神經網絡。 輸入向量的大小等於`vocab_size`，輸出大小對應於類的數量（4）。 因為我們是在解決分類任務，所以最終的激活函數是`LogSoftmax()`。

In [12]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

現在我們將定義標準的 PyTorch 訓練循環。 因為我們的數據集非常大，出於教學目的，我們只會訓練一個 epoch，有時甚至少於一個 epoch（指定 epoch_size 參數允許我們限制訓練）。 我們還會在訓練過程中報告累積的訓練準確率； 報告頻率是使用`report_freq` 參數指定的。

In [13]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [14]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8084375
6400: acc=0.84328125
9600: acc=0.8559375
12800: acc=0.863046875


(0.026000787454373293, 0.8663379530916845)

## BiGrams、TriGrams 和 N-Grams

詞袋方法的一個限制是某些詞是多詞表達的一部分，例如，“熱狗”一詞與其他上下文中的詞“熱”和“狗”具有完全不同的含義。 如果我們總是用相同的向量表示單詞“hot”和“dog”，它會混淆我們的模型。

為了解決這個問題，**N-gram 表示**經常用於文檔分類方法，其中每個詞、雙詞或三詞的頻率是訓練分類器的有用特徵。 例如，在雙元組表示中，除了原始單詞之外，我們還會將所有單詞對添加到詞彙表中。

下面是一個如何使用 Scikit Learn 生成詞表示的二元詞袋的示例：


In [15]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

N-gram 方法的主要缺點是詞彙量開始增長得非常快。 在實踐中，我們需要將 N-gram 表示與一些降維技術相結合，例如 *embeddings*，我們將在下一個單元中討論。

要在我們的 **AG News** 數據集中使用 N-gram 表示，我們需要構建特殊的 ngram 詞彙表：

In [16]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.Vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308844


然後我們可以使用與上面相同的代碼來訓練分類器，但是，它的內存效率非常低。 在下一個單元中，我們將使用嵌入訓練二元分類器。

> **注意：** 您只能保留在文本中出現次數超過指定次數的那些 ngram。 這將確保忽略不常見的二元組，並顯著降低維度。 為此，將 `min_freq` 參數設置為更高的值，並觀察詞彙變化的長度。

## 詞頻逆文檔頻率TF-IDF

在 BoW 表示中，無論單詞本身如何，單詞出現的權重都是均勻的。但是，很明顯，與專業術語相比，*a*、*in* 等頻繁出現的詞對分類的重要性要小得多。事實上，在大多數 NLP 任務中，有些詞比其他詞更相關。

**TF-IDF** 代表**詞頻-逆文檔頻率**。它是詞袋的一種變體，其中使用浮點值而不是指示單詞在文檔中出現的二進制 0/1 值，該值與語料庫中單詞出現的頻率有關。

更正式地，文檔$j$中單詞$i$的權重$w_{ij}$定義為：
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
在哪裡
* $tf_{ij}$是$j$中$i$出現的次數，即我們之前看到的BoW值
* $N$ 是集合中的文檔數
* $df_i$ 是整個集合中包含單詞 $i$ 的文檔數

TF-IDF 值 $w_{ij}$ 與單詞在文檔中出現的次數成正比增加，並被包含該單詞的語料庫中的文檔數所抵消，這有助於調整某些單詞出現的事實比其他人更頻繁。例如，如果該詞出現在集合中的*每個* 文檔中，$df_i=N$ 和 $w_{ij}=0$，這些詞將被完全忽略。

您可以使用 Scikit Learn 輕鬆創建文本的 TF-IDF 矢量化：

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

然而，即使 TF-IDF 表示為不同的單詞提供頻率權重，它們也無法表示含義或順序。 正如著名語言學家 J. R. Firth 在 1935 年所說的那樣，“一個詞的完整意義總是與語境相關的，沒有語境就不能認真研究意義。”。 我們將在後面的單元中學習如何使用語言建模從文本中捕獲上下文信息。
